<a href="https://colab.research.google.com/github/kim-hyun-ho/study/blob/main/210421.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.kaggle.com/c/prudential-life-insurance-assessment
#푸루덴셜 보험 문제